In [2]:
import pandas as pd
import numpy as np

In [3]:
# SEE: https://zenodo.org/record/4632335#.YkGzOShBz4c
filename = '../data/GlobalAlienSpeciesFirstRecordDatabase_v2.xlsx'

df_orig = pd.read_excel(filename, sheet_name=None)
# There are 3 sheets of the excel file
print(df_orig.keys())
df_orig = df_orig['GlobalAlienSpeciesFirstRecordDa']
df_orig.info(show_counts=True)



dict_keys(['ReadMe', 'GlobalAlienSpeciesFirstRecordDa', 'References'])
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61751 entries, 0 to 61750
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   TaxonName         61751 non-null  object
 1   scientificName    61751 non-null  object
 2   Family            57925 non-null  object
 3   Order             60709 non-null  object
 4   Class             60664 non-null  object
 5   Phylum            60793 non-null  object
 6   Taxonomy          57045 non-null  object
 7   OrigName          61751 non-null  object
 8   LifeForm          61571 non-null  object
 9   Region            61751 non-null  object
 10  Island            17552 non-null  object
 11  PresentStatus     33473 non-null  object
 12  FirstRecord       61751 non-null  int64 
 13  FirstRecord_orig  61751 non-null  object
 14  Source            61751 non-null  object
dtypes: int64(1), object(14)
memory us

In [4]:
df = df_orig
# Get the correct one
# df = df[(df['LifeForm'] != 'Insects') & (df['LifeForm'] != 'Vascular plants') & (df['LifeForm'] != 'Fishes') \
#         & (df['LifeForm'] != 'Birds') & (df['LifeForm'] != 'Algae') \
#         & (df['LifeForm'] != 'Invertebrates (excl. Arthropods, Molluscs)') 
#         & (df['LifeForm'] != 'Molluscs') & (df['LifeForm'] != 'Crustaceans') \
#         & (df['LifeForm'] != 'Fungi') & (df['LifeForm'] != 'Reptiles') \
#         # & (df['LifeForm'] != 'Arthropods p.p. (Myriapods, Diplopods etc.)') & (df['LifeForm'] != 'Mammals') \
#         # & (df['LifeForm'] != 'Amphibians') & (df['LifeForm'] != 'Bryophytes') \
#         # & (df['LifeForm'] != 'Amphibians') & (df['LifeForm'] != 'nan') \
#         & (df['LifeForm'] != 'Bryozoa') & (df['LifeForm'] != 'Spiders') \
        # ]
# df = df[(df['FirstRecord'] > 1950) & (df['FirstRecord'] < 2010)]

df = df[df['FirstRecord'] > 1800]

In [5]:
df.info(show_counts=True)
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59256 entries, 0 to 61750
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   TaxonName         59256 non-null  object
 1   scientificName    59256 non-null  object
 2   Family            55551 non-null  object
 3   Order             58228 non-null  object
 4   Class             58181 non-null  object
 5   Phylum            58310 non-null  object
 6   Taxonomy          54696 non-null  object
 7   OrigName          59256 non-null  object
 8   LifeForm          59077 non-null  object
 9   Region            59256 non-null  object
 10  Island            16646 non-null  object
 11  PresentStatus     31458 non-null  object
 12  FirstRecord       59256 non-null  int64 
 13  FirstRecord_orig  59256 non-null  object
 14  Source            59256 non-null  object
dtypes: int64(1), object(14)
memory usage: 7.2+ MB


,TaxonName,scientificName,Family,Order,Class,Phylum,Taxonomy,OrigName,LifeForm,Region,Island,PresentStatus,FirstRecord,FirstRecord_orig,Source
0,Acanthophora muscoides,Acanthophora muscoides (L.) Bory,Rhodomelaceae,Ceramiales,Florideophyceae,Rhodophyta,GBIF,"Acanthophora muscoides Linnaeus, 1753",Algae,Turkey,NaN,NaN,1986,1986,Cinar et al. (2005)
1,Acanthophora nayadiformis,"Acanthophora nayadiformis (Delile) Papenf., 1968",Rhodomelaceae,Ceramiales,Florideophyceae,Rhodophyta,GBIF,Acanthophora nayadiformis,Algae,Cyprus,yes,alien,1997,1997,DAISIE
2,Acanthophora nayadiformis,"Acanthophora nayadiformis (Delile) Papenf., 1968",Rhodomelaceae,Ceramiales,Florideophyceae,Rhodophyta,GBIF,Acanthophora nayadiformis,Algae,Greece,NaN,NaN,1861,1861,AquaNIS (2020)
3,Acanthophora nayadiformis,"Acanthophora nayadiformis (Delile) Papenf., 1968",Rhodomelaceae,Ceramiales,Florideophyceae,Rhodophyta,GBIF,"Acanthophora nayadiformis (Delile) Papenfuss, ...",Algae,Turkey,NaN,NaN,1970,1970,Cinar et al. (2005)
4,Acanthophora spicifera,Acanthophora spicifera (M.Vahl) Børgesen,Rhodomelaceae,Ceramiales,Florideophyceae,Rhodophyta,GBIF,Acanthophora spicifera,Algae,Hawaiian Islands,yes,NaN,1952,1952,Carlton & Eldrege (2009)


In [6]:
family = df['LifeForm'].unique()
region = df['Region'].unique()

print(f'There are {len(family)} families and {len(region)} regions')
species = df['TaxonName'].unique()

print(f'There are {len(species)} species.')
for p in family:
    df_s = df[df['LifeForm'] == p]
    animals = df_s['TaxonName'].unique()
    print(f'{p}[{len(animals)}]', end=', ')


There are 19 families and 276 regions
There are 23039 species.
Algae[546], Amphibians[101], Arthropods p.p. (Myriapods, Diplopods etc.)[166], Bacteria and protozoans[33], Birds[954], Bryophytes[108], Bryozoa[113], Crustaceans[679], Fishes[811], Fungi[464], Insects[5627], Invertebrates (excl. Arthropods, Molluscs)[774], Mammals[254], Molluscs[650], Reptiles[206], Spiders[42], Vascular plants[11362], Viruses[18], nan[0], 

In [7]:
# Filter out all the irrelevant species and regions
# TODO This is really really really slow. A better way to do it?
# Filter out irrelevant data

spec_iter = np.zeros(len(species), dtype=int)
reg_iter = np.zeros(len(region), dtype=int)

for idx, s in enumerate(species):
    df_s = df[df['TaxonName'] == s]
    # n_iter = len(df_s)
    spec_iter[idx] = len(df_s)
    # print(f'The species {s} has {spec_iter[idx]} iteractions')


for idx, s in enumerate(region):
    df_s = df[df['Region'] == s]
    # n_iter = len(df_s)
    reg_iter[idx] = len(df_s)
    # print(f'The species {s} has {spec_iter[idx]} iteractions')

In [8]:
# TODO This is really really really slow. A better way to do it?
# select the species that have less than 3 iteractions
remove_s_idx = np.where(spec_iter < 3)
remove_r_idx = np.where(reg_iter < 10)

# Filter the dataset: remove all rows belonging to species with less than 3 iteractions
for s in species[remove_s_idx]:
    # print(s)  
    df = df.drop(df[df['TaxonName'] == s].index)

# Filter the dataset: remove all rows belonging to region with less than 10 invasions
for s in region[remove_r_idx]:
    # print(s)  
    df = df.drop(df[df['Region'] == s].index)

species_removed = len(species) - len(df['TaxonName'].unique())
print(f'Removed {species_removed} species')
region_removed = len(region) - len(df['Region'].unique())
print(f'Removed {region_removed} region')

# Update region, species
region = df['Region'].unique()
species = df['TaxonName'].unique()


Removed 17574 species
Removed 63 region


In [14]:
# Buld matrix by grouping the dataset into taxonomic families rather than species to reduce the complexity
n_f = len(family)
n_r = len(region)

t_min = df['FirstRecord'].min()
t_max = df['FirstRecord'].max()
print(t_min, t_max, t_max-t_min)
time_res = 10
time = np.arange(t_min, t_max, time_res)

p = n_f + n_r
M = np.zeros((len(time), n_f, n_r))
# M = np.zeros((len(time), p, p)) # if full

for i, t in enumerate(time):
    df_now = df[(df['FirstRecord'] >= t) & (df['FirstRecord'] < t+(time_res-1))]
    for index, row in df_now.iterrows():
        f = row['LifeForm']
        r = row['Region']
        # print(f'Family {f} invaded region {r} at time {t}')

        f_idx = np.where(family == f)
        r_idx = np.where(region == r)
        M[i, f_idx, r_idx] = 1
M.shape

1801 2020 219


(22, 19, 276)

In [9]:
# Buld matrix
n_s = len(species)
n_r = len(region)

t_min = df['FirstRecord'].min()
t_max = df['FirstRecord'].max()
print(t_min, t_max, t_max-t_min)
time_res = 10
time = np.arange(t_min, t_max, time_res)

p = n_s + n_r
M = np.zeros((len(time), n_s, n_r))
# M = np.zeros((len(time), p, p)) # if full

for i, t in enumerate(time):
    df_now = df[(df['FirstRecord'] >= t) & (df['FirstRecord'] < t+(time_res-1))]
    for index, row in df_now.iterrows():
        s = row['TaxonName']
        r = row['Region']
        # print(f'Species {s} invaded region {r} at time {t}')

        s_idx = np.where(species == s)
        r_idx = np.where(region == r)
        M[i, s_idx, r_idx] = 1
M.shape

1801 2020 219


(22, 5465, 213)

In [10]:
# Export data and then import it into R
print(n_s, n_r)
M = M.reshape(len(time), n_s*n_r)
# M = M.reshape(len(time), (p)**2) # if full matix
print(f's = {n_s}, r = {n_r}')
print(M.shape)
np.save('../data/matrix.npy', M)

5465 213
s = 5465, r = 213
(22, 1164045)


In [2]:
import numpy as np
# load matrix
M = np.load('../data/matrix.npy')

In [3]:
print(f'There are in total {np.sum(M)} iteraction.')
print(M[:10])

There are in total 32648.0 iteraction.
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [4]:
# Load matrix into CUDA and see how much space it's using
import cupy as cp
mempool = cp.get_default_memory_pool()
print(mempool.used_bytes())              # 0
print(mempool.total_bytes())             # 0
M_gpu = cp.array(M)
est_bytes = np.prod(M.shape)*8
print(f'My estimate {est_bytes}') # num bytes
print(M.nbytes)                          # ~ num bytes
print(mempool.used_bytes())              # ~ num bytes
print(mempool.total_bytes())             # ~ num bytes

print(f'In MB {est_bytes/(10**6)}')

0
0
My estimate 204871920
204871920
204872192
204872192
In MB 204.87192


In [19]:
# TODO we should use sparse matrices for the data

import scipy.sparse

sp = scipy.sparse.csr_matrix(M)

In [47]:
import cupy as cp
from cupyx.scipy.sparse import csr_matrix as csr_gpu
from scipy.sparse import random

sp = random(22, n_s*n_r, density=0.25)
sp_gpu = csr_gpu(sp)
x = np.random.rand(n_s*n_r)

res = sp.dot(x)

print(sp_gpu.shape, x.shape)
x_gpu = cp.array(x) #moving x to the gpu
print(sp_gpu.shape[1] != len(x_gpu))

res_gpu = sp_gpu.dot(x_gpu)

if np.allclose(res, cp.asnumpy(res_gpu)):
    print("They are the same!")
else:
    print("They are different!")

(22, 5244) (5244,)
False
They are the same!
